# Geographic Distribution of Hate Crimes in California (2012–2022)

### Introduction
This page explores the geographic distribution of hate crimes across California's counties from 2012 to 2022. An interactive map allows users to visualize the hate crime counts for each county by selecting a specific year.

Hate crime data includes incidents motivated by race, ethnicity, religion, sexual orientation, gender, and other factors. The visualizations aim to highlight regional disparities and trends over time.

### Loading Geographic Data

This section loads California county boundaries from the `CA_Counties.shp` shapefile using `geopandas`. The shapefile contains geographic features (county shapes) and attribute data (e.g., county names), which are stored in a GeoDataFrame (`gdf`) for mapping and spatial analysis.


In [4]:
import geopandas as gpd

# Specify the path to the .shp file
shapefile_path = "C:/Users/Ilike/CSCI385/Final_Project/ca_counties/CA_Counties.shp"

# Load the shapefile into a Geopandas DataFrame
gdf = gpd.read_file(shapefile_path)

### Processing Hate Crime Data for Geographic Analysis

This function processes hate crime data for a specific year to prepare it for geographic visualization. It ensures that all counties are represented, even if they report zero hate crimes for the given year.

#### Steps Explained:
1. **Load the Datasets**:
   - `Hate Crime 2012 - 2022.csv`: Contains hate crime incidents across California.
   - `NCIC Code Jurisdiction List.csv`: Maps numeric county codes to county names.

2. **Filter for Specific Bias Type**:
   - Select hate crimes motivated by "Race/Ethnicity/Ancestry" from the dataset.

3. **Merge Datasets**:
   - Combine the hate crime data with the NCIC jurisdiction data using the numeric `County` code.

4. **Group and Aggregate Data**:
   - Group the merged data by county and year (`ClosedYear`) to count the total number of hate crimes per county.

5. **Filter by Year**:
   - Extract data for the specified year to focus on a single time slice.

6. **Ensure All Counties are Represented**:
   - Merge the yearly data with the complete list of counties from the NCIC data to include counties with zero hate crimes for the year.

7. **Handle Missing Values**:
   - Replace `NaN` values in the hate crime count with `0`.
   - Fill missing years with the specified year to standardize the dataset.

8. **Convert Data Types**:
   - Convert `HateCrimeCount` and `ClosedYear` columns to integers for consistency.

9. **Reset Index**:
   - Reset the index for a clean, tabular output.

10. **View the Result**:
   - Print the resulting dataset for verification.

#### Example Usage:
- The function is called for the year 2013:
  ```python
  process_hate_crime_data(2013)


In [5]:
import pandas as pd

def process_hate_crime_data(year):
    # Load the datasets
    hate_crime_data = pd.read_csv('Hate Crime 2012 - 2022.csv')
    ncic_data = pd.read_csv('NCIC Code Jurisdiction List.csv')

    # Step 1: Filter the data for 'Race/Ethnicity/Ancestry' bias type
    race_ethnicity_ancestry_data = hate_crime_data[hate_crime_data['MostSeriousBiasType'] == 'Race/Ethnicity/Ancestry']

    # Step 2: Merge the datasets based on the numeric County code and CntyCode
    merged_data = pd.merge(race_ethnicity_ancestry_data, ncic_data, left_on='County', right_on='CntyCode', how='left')

    # Step 3: Group the merged data by 'County' and 'ClosedYear', and count the occurrences (hate crimes)
    county_yearly_hate_crimes = merged_data.groupby(['County_y', 'ClosedYear']).agg({'RecordId': 'count'}).reset_index()

    # Step 4: Rename columns for clarity
    county_yearly_hate_crimes.rename(columns={'RecordId': 'HateCrimeCount', 'County_y': 'County'}, inplace=True)

    # Step 5: Filter data for the specified year
    hate_crimes_year = county_yearly_hate_crimes[county_yearly_hate_crimes['ClosedYear'] == year]

    # Step 6: Merge with NCIC data to ensure all counties are included, even those without data
    all_counties_year = pd.merge(ncic_data, hate_crimes_year, how='left', left_on='County', right_on='County')

    # Step 7: Replace NaN values in HateCrimeCount with 0 for counties with no data and set ClosedYear to the selected year
    all_counties_year['HateCrimeCount'] = all_counties_year['HateCrimeCount'].fillna(0)
    all_counties_year['ClosedYear'] = all_counties_year['ClosedYear'].fillna(year)

    # Step 8: Convert HateCrimeCount and ClosedYear to integers (removes decimals)
    all_counties_year['HateCrimeCount'] = all_counties_year['HateCrimeCount'].astype(int)
    all_counties_year['ClosedYear'] = all_counties_year['ClosedYear'].astype(int)

    # Step 9: Reset index for clarity
    all_counties_year = all_counties_year.reset_index(drop=True)

    # Step 10: View the resulting data
    print(all_counties_year)

# Example: Run for the year 2013
process_hate_crime_data(2013)


    CntyCode                  County  ClosedYear  HateCrimeCount
0          1          Alameda County        2013              36
1          2           Alpine County        2013               1
2          3           Amador County        2013               0
3          4            Butte County        2013               3
4          5        Calaveras County        2013               1
5          6           Colusa County        2013               0
6          7     Contra Costa County        2013              16
7          8        Del Norte County        2013               0
8          9        El Dorado County        2013               1
9         10           Fresno County        2013               7
10        11            Glenn County        2013               0
11        12         Humboldt County        2013               1
12        13         Imperial County        2013               0
13        14             Inyo County        2013               0
14        15             

### Visualizing Hate Crimes Across California Counties

This section creates an interactive map to visualize hate crimes in California by county for each year from 2012 to 2022.

#### **Steps:**
1. **Load Data**:
   - Hate crime data (`Hate Crime 2012 - 2022.csv`), jurisdiction mapping (`NCIC Code Jurisdiction List.csv`), and California county shapefile (`CA_Counties.shp`).

2. **Preprocess Data**:
   - Filter for crimes based on **Race/Ethnicity/Ancestry**.
   - Merge hate crime data with county information.
   - Group by county and year to calculate total hate crimes.

3. **Interactive Map**:
   - The `visualize_hate_crime_year(year)` function generates a map showing hate crimes by county for the selected year:
     - Counties are color-coded (darker = more crimes).
     - A slider lets users choose the year (2012–2022).
     - The map updates dynamically based on the selection.


In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from ipywidgets import interact

# Load and process the data once
hate_crime_data = pd.read_csv('Hate Crime 2012 - 2022.csv')
ncic_data = pd.read_csv('NCIC Code Jurisdiction List.csv')
shapefile_path = r'C:\Users\Ilike\CSCI385\Final_Project\ca_counties\CA_Counties.shp'
ca_counties = gpd.read_file(shapefile_path)

# Preprocess data
race_ethnicity_ancestry_data = hate_crime_data[hate_crime_data['MostSeriousBiasType'] == 'Race/Ethnicity/Ancestry']
merged_data = pd.merge(race_ethnicity_ancestry_data, ncic_data, left_on='County', right_on='CntyCode', how='left')
county_yearly_hate_crimes = merged_data.groupby(['County_y', 'ClosedYear']).agg({'RecordId': 'count'}).reset_index()
county_yearly_hate_crimes.rename(columns={'RecordId': 'HateCrimeCount', 'County_y': 'County'}, inplace=True)

def visualize_hate_crime_year(year):
    # Filter data for the selected year
    hate_crimes_year = county_yearly_hate_crimes[county_yearly_hate_crimes['ClosedYear'] == year]
    all_counties_year = pd.merge(ncic_data, hate_crimes_year, how='left', left_on='County', right_on='County')
    all_counties_year['HateCrimeCount'] = all_counties_year['HateCrimeCount'].fillna(0).astype(int)

    # Merge with shapefile data
    merged_geo = ca_counties.merge(all_counties_year, left_on='NAMELSAD', right_on='County', how='left')
    merged_geo['HateCrimeCount'] = merged_geo['HateCrimeCount'].fillna(0).astype(int)

    # Plot the data
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    merged_geo.plot(column='HateCrimeCount',
                    cmap='OrRd',
                    linewidth=0.8,
                    ax=ax,
                    edgecolor='0.8',
                    legend=True)
    ax.set_title(f'Hate Crime Count by County in California ({year})', fontsize=16)
    ax.axis('off')

    # Explicitly display the figure
    plt.show()

    # Clear the figure to prevent implicit display
    plt.close(fig)

# Interactive slider
interact(visualize_hate_crime_year, year=(2012, 2022))


interactive(children=(IntSlider(value=2017, description='year', max=2022, min=2012), Output()), _dom_classes=(…

<function __main__.visualize_hate_crime_year(year)>

![Interactive Hate Crime Map](HateCrime_Interactive.gif)